# Getting started

The `pivmetalib` implements many (but not all) classes for various ontologies and vocabularies used in the ontology ["pivmeta"](https://matthiasprobst.github.io/pivmeta/).

Let's find out, what that means for the description of a Person (A person can be described by the PROV namespace: https://www.w3.org/ns/prov)

In [1]:
from pivmetalib import prov

In [2]:
creator = prov.Person(
    lastName='Okamoto',
    mbox="okamoto@tokai.t.u-tokyo.ac.jp"
)
creator

Person(mbox=okamoto@tokai.t.u-tokyo.ac.jp, lastName=Okamoto)

**What's behind the class:** Each class is inherited from the `pydantic.BaseModel`. The package [`pydantic`](https://docs.pydantic.dev/latest/) allows validating the attributes of a class.

Here's a minimal example of how such a class is built:

In [3]:
import pydantic

class MyPerson(pydantic.BaseModel):
    name: str
    age: int
    website: pydantic.HttpUrl=None

jd = MyPerson(name='John Doe', age=34)
jd

MyPerson(name='John Doe', age=34, website=None)

See, very simple but effective. Let's provoke an error by passing an invalid website URL:

In [4]:
try:
    MyPerson(name='John Doe', age=34, website='invalid.de')
except pydantic.ValidationError as e:
    print(e)

1 validation error for MyPerson
website
  Input should be a valid URL, relative URL without a base [type=url_parsing, input_value='invalid.de', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/url_parsing


We could improve the model by using other built-in validators, like `PositiveInt` to ensure, that age is always positive:

In [5]:
class MyPerson(pydantic.BaseModel):
    name: str
    age: pydantic.PositiveInt
    website: pydantic.HttpUrl=None

In [6]:
try:
    MyPerson(name='John Doe', age=-4)
except pydantic.ValidationError as e:
    print(e)

1 validation error for MyPerson
age
  Input should be greater than 0 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/greater_than


## Example: prov.Person

The classes in the `pivmetalib` namespace modules like `prov` are like that *and* moreover, include some other helpful methods.

The easiest way to find out, which fields are implemented, call the property `model_field`:

In [7]:
prov.Person.model_fields.keys()

dict_keys(['id', 'label', 'mbox', 'firstName', 'lastName', 'hadRole', 'wasRoleIn'])

Note, that you can add fields, that are note pre-defined, too.

In [8]:
creator = prov.Person(
    firstName='John',
    lastName='Doe',
    nickname='JD'
)
creator

Person(firstName=John, lastName=Doe, nickname=JD)

## Working with the classes

There are two importing things, you can do with the classes:
- **Dumping**: This means exporting the content to various formats (dictionary, json-ld strings, ...)
- **querying**: Finding data of a class in a JSON-LD file

### Dumping:
The most basic form of "dumping" is exporting the class instance to a dictionary:

In [9]:
creator.model_dump()

{'id': None,
 'label': None,
 'mbox': None,
 'firstName': 'John',
 'lastName': 'Doe',
 'hadRole': None,
 'wasRoleIn': None,
 'nickname': 'JD'}

The `pivmetalib` adds also `dump_jsonld` to the classes and allows to create a JSON-LD string, which can be saved to a file, too:

In [10]:
print(creator.dump_jsonld())

[
    {
        "@id": "https://local-domain.org/08b47721-4927-40be-b802-bcd1933892bd",
        "@type": [
            "prov:Person"
        ],
        "foaf:firstName": [
            {
                "@value": "John"
            }
        ],
        "foaf:lastName": [
            {
                "@value": "Doe"
            }
        ]
    }
]


**Save JSON-LD file**

In [11]:
import pivmetalib

In [12]:
# save to file:
with open('john.jsonld', 'w') as f:
    creator.dump_jsonld(context=pivmetalib.CONTEXT)

Let's create another person and save both to the file. For this we will need to "merge" both JSON-LD strings. There is a helper function for this in the `utils` module: 

In [13]:
other_person = prov.Person(first_name='Lisa')

In [14]:
from pivmetalib import utils

In [15]:
# save to file:
with open('creator.jsonld', 'w') as f:
    f.write(utils.merge_jsonld([creator.dump_jsonld(context=pivmetalib.CONTEXT), other_person.dump_jsonld(context=pivmetalib.CONTEXT)]))

### Querying

We can identify data within a JSON-LD file by calling `query` and providing the class we want to find. The method applies a SPARQL query ad returns an instance of the provided object:

In [16]:
from ontolutils import query

In [17]:
persons = query(cls=prov.Person, source='creator.jsonld', context=pivmetalib.CONTEXT)

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX local: <http://example.com/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT *
WHERE {{
    ?id a prov:Person .
    ?id ?p ?o .
}}


As expected, we find both persons:

In [18]:
import rdflib

In [19]:
g = rdflib.Graph().parse(source='creator.jsonld', context=pivmetalib.CONTEXT)

In [20]:
for s,p,o in g:
    print(s, o)

https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a http://www.w3.org/ns/prov#Person
https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a John
https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a JD
https://local-domain.org/d78fbcc9-d9f7-4704-9dee-3c1fd7e4f234 http://www.w3.org/ns/prov#Person
https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a Doe
https://local-domain.org/d78fbcc9-d9f7-4704-9dee-3c1fd7e4f234 Lisa


In [21]:
qs = """PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX local: <http://example.com/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT *
WHERE {
    ?id a prov:Person .
    ?id ?p ?o .
}"""

In [22]:
for r in g.query(qs):
    print(r)

(rdflib.term.URIRef('https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a'), rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a'), rdflib.term.Literal('John'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/firstName'))
(rdflib.term.URIRef('https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a'), rdflib.term.Literal('Doe'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/lastName'))
(rdflib.term.URIRef('https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a'), rdflib.term.Literal('JD'), rdflib.term.URIRef('https://matthiasprobst.github.io/pivmeta#nickname'))
(rdflib.term.URIRef('https://local-domain.org/d78fbcc9-d9f7-4704-9dee-3c1fd7e4f234'), rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('https://local-domain.

In [23]:
for person in persons:
    print(person.model_dump(exclude_none=True))

{'id': 'https://local-domain.org/4a433b7f-d1dd-4d47-a5de-80db5b84d15a', 'firstName': 'John', 'lastName': 'Doe', 'type': {}, 'nickname': 'JD'}
{'id': 'https://local-domain.org/d78fbcc9-d9f7-4704-9dee-3c1fd7e4f234', 'type': {}, 'first_name': 'Lisa'}
